Dans ce TD, nous allons implémenter une collecte simple de données via une API : https://rapidapi.com/edamam/api/edamam-food-and-grocery-database. Cela permet d'enrichir des jeux de données d'images lorsque notre problématique requiert par exemple des images spécifiques.

Le but est d'obtenir une extraction des produits à base de champagne dans un fichier “.csv”, contenant pour chaque produit les données suivantes : foodId, label, category, foodContentsLabel, image, ainsi que les images correspondantes.

Rassembler les imports dans la cellule suivante.

In [3]:
import os

from dotenv import load_dotenv
import pandas as pd
import requests

La communication avec une API implique de s'identifier. Cela se fait par l'intermédiaire d'une clef. C'est une bonne pratique de ne pas laisser tout ce qui touche à l'identification en dur dans le code si celui-ci est amené à être partagé. Il existe plusieurs manières d'éviter cet écueil, en voici un bon résumé : https://blog.gitguardian.com/how-to-handle-secrets-in-python/.

Ici, pour accélérer la mise en place et ne pas vous faire créer un compte, je vous partage une clef API d'un compte test. Pour autant, nous suivrons le principe de séparer la clef du code en utilisant la méthode 1 du résumé, à savoir par l'intermédiaire d'un fichier .env. ainsi que la librairie dotenv (nom du module), qui nécessitera une installation (nom du package : python-dotenv).



Récupérer les variables d'environnement du fichier .env en appelant load_dotenv(). Utiliser os.getenv pour stocker la valeur de la clef API dans une variable. Vérifier TEMPORAIREMENT que la clef a bien été chargée en l'affichant.

In [4]:
load_dotenv()
API_KEY = os.getenv("API_KEY")

En observant la requête exemple dans "Code Snippets", préparer les champs d'une requête permettant de récolter les produits appartenant au type de nutrition "cooking" et contenant au moins l'ingrédient "champagne". Trouver le nommage exact des paramètres à renseigner dans la requête grâce à la documentation des paramètres optionnels de l'API.

In [6]:
url = "https://edamam-food-and-grocery-database.p.rapidapi.com/api/food-database/v2/parser"

headers = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": "edamam-food-and-grocery-database.p.rapidapi.com",
}

params = {
    "nutrition-type": "cooking",
    "ingr": "champagne",
}

A l'aide de la bibliothèque requests, envoyer la requête GET correspondante à l'API. Stocker l'output dans une variable et en faire un affichage.

In [11]:
response = requests.get(url, params, headers=headers)

La fonction print renvoie le code de la réponse, permettant de confirmer qu'elle fut un succès ou qu'elle a rencontré une erreur. Accéder désormais au contenu de la réponse en utilisant la méthode .json().

In [13]:
response.json()

{'text': 'champagne',
 'count': 295,
 'parsed': [{'food': {'foodId': 'food_a656mk2a5dmqb2adiamu6beihduu',
    'uri': 'http://www.edamam.com/ontologies/edamam.owl#Food_table_white_wine',
    'label': 'Champagne',
    'knownAs': 'dry white wine',
    'nutrients': {'ENERC_KCAL': 82.0,
     'PROCNT': 0.07,
     'FAT': 0.0,
     'CHOCDF': 2.6,
     'FIBTG': 0.0},
    'category': 'Generic foods',
    'categoryLabel': 'food',
    'image': 'https://www.edamam.com/food-img/a71/a718cf3c52add522128929f1f324d2ab.jpg'}}],
 'hints': [{'food': {'foodId': 'food_a656mk2a5dmqb2adiamu6beihduu',
    'uri': 'http://www.edamam.com/ontologies/edamam.owl#Food_table_white_wine',
    'label': 'Champagne',
    'knownAs': 'dry white wine',
    'nutrients': {'ENERC_KCAL': 82.0,
     'PROCNT': 0.07,
     'FAT': 0.0,
     'CHOCDF': 2.6,
     'FIBTG': 0.0},
    'category': 'Generic foods',
    'categoryLabel': 'food',
    'image': 'https://www.edamam.com/food-img/a71/a718cf3c52add522128929f1f324d2ab.jpg'},
   'measur

Reconstruire le dataframe correspondant aux données qui nous intéressent (pour rappel, seuls les champs suivants nous intéressent : foodId, label, category, foodContentsLabel, image).

In [26]:
wanted_cols = ["foodId", "label", "category", "foodContentsLabel", "image"]
data=[
        {
            col: hint["food"].get(col)
            for col in wanted_cols
        }
        for hint in response.json()["hints"]
    ]

champagne_df = pd.DataFrame(
    data=data,
    columns=wanted_cols,
)

In [28]:
champagne_df.head()

,foodId,label,category,foodContentsLabel,image
0,food_a656mk2a5dmqb2adiamu6beihduu,Champagne,Generic foods,None,https://www.edamam.com/food-img/a71/a718cf3c52...
1,food_b8d1c2rbdpdok6bjpwmtlbsa5hs4,"Champagne Vinaigrette, Champagne",Packaged foods,OLIVE OIL; BALSAMIC VINEGAR; CHAMPAGNE VINEGAR...,None
2,food_axmullnbxsm8f5aug13l2aqoyu6j,"Champagne Vinaigrette, Champagne",Packaged foods,INGREDIENTS: WATER; CANOLA OIL; CHAMPAGNE VINE...,https://www.edamam.com/food-img/d88/d88b64d973...
3,food_a7j75b0bk4sd87a34ed3tats8n31,"Champagne Dressing, Champagne",Packaged foods,OLIVE OIL; BALSAMIC VINEGAR; CHAMPAGNE VINEGAR...,None
4,food_b2xjyr5beqff2mbwycgjza8o1hxk,"Champagne Vinaigrette, Champagne",Packaged foods,CANOLA AND SOYBEAN OIL; WHITE WINE (CONTAINS S...,None


Afficher les 10 premiers éléments du dataframe ainsi que sa taille totale.

In [ ]:
# Votre code ici

Pour certaines lignes, l'image n'est pas disponible et dans ce cas l'URL est à None. Afficher le nombre de lignes dont l'image dispose d'une URL.

In [29]:
champagne_df["image"].count()

np.int64(4)

In [ ]:
# Votre code ici

Enregistrer cette extraction au format csv dans le dossier courant.

In [ ]:
# Votre code ici

Finalement, récupérer les images dont l'URL est renseignée, à l'aide de .content. Les sauvegarder dans le dossier "images". Vérifier que le nombre de fichiers téléchargés correspond bien au nombre d'images attendu.

In [30]:
num = 0
for url in champagne_df["image"]:
    if not pd.isnull(url):
        img_responses = requests.get(url).content
        with open(os.path.join("../../data/champagne", f"champagne_{num}.jpg"), "wb") as file:
            file.write(img_responses)
        num += 1